In [63]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import model_from_json

from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

import pandas as pd
import numpy as np
import os

In [64]:
#Everything I need from the other notebook to load
time = pd.read_csv("/content/drive/My Drive/Crypto/time.csv")
y_test = pd.read_csv(r"/content/drive/My Drive/Crypto/y_test.csv")
LGR_model = pickle.load(open("/content/drive/My Drive/Crypto/CryptoLGR_model.sav", "rb"))
LGR2_model = pickle.load(open("/content/drive/My Drive/Crypto/CryptoLGR2_model.sav", "rb"))
LGR3_model = pickle.load(open("/content/drive/My Drive/Crypto/CryptoLGR3_model.sav", "rb"))
LGR4_model = pickle.load(open("/content/drive/My Drive/Crypto/CryptoLGR4_model.sav", "rb"))

XBG_model = pickle.load(open("/content/drive/My Drive/Crypto/XGBC_model.sav", "rb"))
XBG2_model = pickle.load(open("/content/drive/My Drive/Crypto/XGBC2_model.sav", "rb"))
XBG3_model = pickle.load(open("/content/drive/My Drive/Crypto/XGBC3_model.sav", "rb"))
XBG4_model = pickle.load(open("/content/drive/My Drive/Crypto/XGBC4_model.sav", "rb"))

lstm_df = pd.read_csv(r"/content/drive/My Drive/Crypto/lstm_df.csv")
lstm_model = keras.models.load_model("/content/drive/My Drive/Crypto/lstm_model.h5")
lstm_model.load_weights("/content/drive/My Drive/Crypto/lstm_model_weights.h5")


LGR_predictions = np.loadtxt("/content/drive/My Drive/Crypto/LGR_predictions.txt")
XGB_predictions = np.loadtxt("/content/drive/My Drive/Crypto/XGB_predictions.txt")
LSTM_predictions = np.loadtxt("/content/drive/My Drive/Crypto/LSTM_X.txt")

pca2 = np.loadtxt("/content/drive/My Drive/Crypto/LGR2_X.txt")
pca3 = np.loadtxt("/content/drive/My Drive/Crypto/LGR3_X.txt")
pca4 = np.loadtxt("/content/drive/My Drive/Crypto/LGR4_X.txt")

xpca2 = np.loadtxt("/content/drive/My Drive/Crypto/XGB2_predictions.txt")
xpca3 = np.loadtxt("/content/drive/My Drive/Crypto/XGB3_predictions.txt")
xpca4 = np.loadtxt("/content/drive/My Drive/Crypto/XGB4_predictions.txt")

In [27]:
#This is pretty low!
print("Log regression classification report is: ","\n", classification_report(y_test,LGR_predictions))

Log regression classification report is:  
               precision    recall  f1-score   support

           0       0.52      0.68      0.59    103456
           1       0.56      0.40      0.47    106784

    accuracy                           0.54    210240
   macro avg       0.54      0.54      0.53    210240
weighted avg       0.54      0.54      0.53    210240



My ram isn't enough, please uncomment the code for the visual. Thank you

I also added a link

In [4]:
# # Link: "https://github.com/AsterLaoWhy/Thinkful/blob/master/Target%20and%20predictions.png"
#sns.heatmap(confusion_matrix(y_test,LGR_predictions), cmap = "Blues", annot=True)

In [65]:
#... 
print("XGB regression classification report is: ","\n", classification_report(y_test,XGB_predictions))

XGB regression classification report is:  
               precision    recall  f1-score   support

           0       0.49      0.45      0.47    103456
           1       0.50      0.54      0.52    106784

    accuracy                           0.49    210240
   macro avg       0.49      0.49      0.49    210240
weighted avg       0.49      0.49      0.49    210240



My ram isn't enough, please uncomment the code for the visual. Thank you

In [ ]:
# # Link: "https://github.com/AsterLaoWhy/Thinkful/blob/master/XGB%20Heatmap.png"
# sns.heatmap(confusion_matrix(y_test,XGB_predictions), cmap = "Blues", annot=True)

In [7]:
#Again, this is to make X,Y,and format it for LSTM models 
def convert_data(data, step):
  X, y = [], []
  for i in range(len(data)-step):
    d = i + step
    X.append(data[i:d,:-1])
    y.append(data[d,-1])
  return np.array(X), np.array(y)

In [29]:
time.drop(columns = 'time', inplace = True)
lstm_df = lstm_df.join(time, how = "left")
#resize to fit predictions
lstm_df = lstm_df[-210240:]
lstm_df['LGR_predictions'] = LGR_predictions
lstm_df['XGB_predictions'] = XGB_predictions

In [ ]:
#amont in 1 week
#Link:
# plt.scatter(x="time.1", y = "target", data=lstm_df[-10080:])
# plt.scatter(x="time.1", y = "LGR_predictions", data=lstm_df[-10080:])
# plt.legend()
# plt.show()

In [ ]:
#amount in 1 week
# # Link: "https://github.com/AsterLaoWhy/Thinkful/blob/master/XBG%20Predictions.png"
# plt.scatter(x="time.1", y = "target", data=lstm_df[-10080:])
# plt.scatter(x="time.1", y = "XGB_predictions", data=lstm_df[-10080:])
# plt.legend()
# plt.show()

In [30]:
# #Show amount earned based on LGR predictions
# #I'm not a wealthy man

def backtesting(prediction_column):


  one_week_df = lstm_df[-10080:]
  potential_initial_investment = 1000
  potential_money_earned = 0
  potential_holdings = 0
  potential_status = 0
  actual_initial_investment = 1000
  actual_money_earned = 0
  actual_holdings = 0
  actual_status = 0
  # 0 Statusmeans we are not holding money. If status is 1, we are invested

  for i in range(len(one_week_df)):
    if (one_week_df.iloc[i]['target']==1 and potential_status ==0):
      amount_invested = potential_initial_investment*.3
      potential_holdings = (amount_invested)/(one_week_df.iloc[i]['close'])
      potential_status = 1
      potential_initial_investment = potential_initial_investment-amount_invested

    elif (one_week_df.iloc[i]['target']==0 and potential_status ==1):
      potential_money_earned = potential_holdings*one_week_df.iloc[i]['close']
      potential_initial_investment += potential_money_earned
      potential_status = 0
    else:
      pass


  for i in range(len(one_week_df)):
    if (one_week_df.iloc[i][prediction_column]==1 and actual_status == 0):
      amount_invested = actual_initial_investment*.3
      actual_holdings = (amount_invested)/(one_week_df.iloc[i]['close'])
      actual_status = 1
      actual_initial_investment = actual_initial_investment-amount_invested

    elif (one_week_df.iloc[i][prediction_column]==0 and actual_status == 1):
      actual_money_earned = actual_holdings*one_week_df.iloc[i]['close']
      actual_initial_investment += actual_money_earned
      actual_status = 0
    else:
      pass

  print(f"strategy optimally earns: ${potential_initial_investment} in one weeks")
  print(f"{prediction_column} strategy earns: ${actual_initial_investment} in one week")

In [31]:
#LGR backtesting
backtesting("LGR_predictions")

strategy optimally earns: 1070.9543749525112 in one weeks
LGR_predictions strategy earns: 982.9253753442499 in one week


In [32]:
#XBG bactesting
backtesting("XGB_predictions")


strategy optimally earns: 1070.9543749525112 in one weeks
XGB_predictions strategy earns: 694.4513957306814 in one week


In [48]:
#There's more LSTM predictions so I'll resize and backtest
lstm_df["LSTM_predictions"] = LSTM_predictions[52499:]
backtesting("LSTM_predictions")

strategy optimally earns: 1070.9543749525112 in one weeks
LSTM_predictions strategy earns: 700.0 in one week


In [49]:
#Well poop. Let's see if the unsupervised learning helped at all

In [50]:
y_test = y_test[-210240:]

In [51]:
#PCA doesn't really do anything for Log regression, let's see if it does something for XGB boosting
print("PCA 2 Log regression classification report is: ","\n", classification_report(y_test,pca2))
print("PCA 3 Log regression classification report is: ","\n", classification_report(y_test,pca3))
print("PCA 4 Log regression classification report is: ","\n", classification_report(y_test,pca4))

PCA 2 Log regression classification report is:  
               precision    recall  f1-score   support

           0       0.48      0.35      0.40    103456
           1       0.50      0.64      0.56    106784

    accuracy                           0.50    210240
   macro avg       0.49      0.49      0.48    210240
weighted avg       0.49      0.50      0.48    210240

PCA 3 Log regression classification report is:  
               precision    recall  f1-score   support

           0       0.53      0.53      0.53    103456
           1       0.55      0.55      0.55    106784

    accuracy                           0.54    210240
   macro avg       0.54      0.54      0.54    210240
weighted avg       0.54      0.54      0.54    210240

PCA 4 Log regression classification report is:  
               precision    recall  f1-score   support

           0       0.53      0.53      0.53    103456
           1       0.55      0.55      0.55    106784

    accuracy                    

In [52]:
#Not really super helpful
print("PCA 2 XGB regression classification report is: ","\n", classification_report(y_test,xpca2))
print("PCA 3 XGB regression classification report is: ","\n", classification_report(y_test,xpca3))
print("PCA 4 XGB regression classification report is: ","\n", classification_report(y_test,xpca4))

PCA 2 XGB regression classification report is:  
               precision    recall  f1-score   support

           0       0.48      0.25      0.33    103456
           1       0.50      0.74      0.60    106784

    accuracy                           0.50    210240
   macro avg       0.49      0.49      0.46    210240
weighted avg       0.49      0.50      0.46    210240

PCA 3 XGB regression classification report is:  
               precision    recall  f1-score   support

           0       0.53      0.44      0.48    103456
           1       0.53      0.62      0.58    106784

    accuracy                           0.53    210240
   macro avg       0.53      0.53      0.53    210240
weighted avg       0.53      0.53      0.53    210240

PCA 4 XGB regression classification report is:  
               precision    recall  f1-score   support

           0       0.53      0.47      0.50    103456
           1       0.54      0.60      0.57    106784

    accuracy                    

In [55]:
#See if XPCA 3 is better since recal isn't terrible
lstm_df = lstm_df[-210240:]
lstm_df['Lpca2'] = pca2
lstm_df['Lpca3'] = pca3
lstm_df['Lpca4'] = pca4
lstm_df['Xpca2'] = xpca2
lstm_df['Xpca3'] = xpca3
lstm_df['Xpca4'] = xpca4


In [56]:
backtesting("Lpca2")

strategy optimally earns: 1070.9543749525112 in one weeks
Lpca2 strategy earns: 700.0 in one week


In [57]:
backtesting("Lpca3")

strategy optimally earns: 1070.9543749525112 in one weeks
Lpca3 strategy earns: 970.2319944035494 in one week


In [58]:
#THIS IS THE BEST FOR PCA IN LOG
backtesting("Lpca4")

strategy optimally earns: 1070.9543749525112 in one weeks
Lpca4 strategy earns: 974.8091453753588 in one week


In [60]:
backtesting("Xpca2")

strategy optimally earns: 1070.9543749525112 in one weeks
Xpca2 strategy earns: 698.2293654446223 in one week


In [61]:
backtesting("Xpca3")

strategy optimally earns: 1070.9543749525112 in one weeks
Xpca3 strategy earns: 968.7105187531038 in one week


In [62]:
#THIS IS THE BEST FOR XGBOOST PCA
backtesting("Xpca4")

strategy optimally earns: 1070.9543749525112 in one weeks
Xpca4 strategy earns: 979.6390404584286 in one week
